# Aim of notebook:
To generate the relations or kinships data

In [41]:
%env CUDA_VISIBLE_DEVICES=0

In [205]:
TRANSFORMERS_CACHE='/pool.ssd/assets/hub'

In [43]:
datetime_format = '%Y-%m-%d_%H:%M:%S'

In [44]:
%load_ext autoreload
%autoreload 2

In [244]:
import torch
import transformers
import tqdm
import ast

# pip install accelerate

from transformers import AutoProcessor, Gemma3ForConditionalGeneration
# import requests
import datetime as dt
import tqdm
import re

In [46]:
# !ls /assets/models/

In [47]:
# %pip install git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3


In [48]:
# from huggingface_hub import login
# login()

In [218]:
# model_id = "google/gemma-3-27b-it"
# model_ids_gemma = ["google/gemma-3-4b-it", "google/gemma-3-12b-it", "google/gemma-3-27b-it"]
# model_paths_gemma = ['/assets/models/google-gemma-3-it-4b', '/assets/models/google-gemma-3-it-12b', '/assets/models/google-gemma-3-it-27b']
# model_path = "/assets/models/google-gemma-3-it-27b"


# for model_path in model_ids_gemma:
#     tokenizer, model, processor = model_init(model_path)
#     print("model loaded")
#     print("model_path", model_path)


# model_ids_gemma=[  'google/gemma-3-12b-it' ]
# PROMPT_TYPE = 'all_gender' # 'all_gender' or 'female' are so far two options
# TGT_LANG = 'Hindi' #'Hindi' # for prompt to generate data

# TGT_LANG_CODE = 'hin' #'hin' # to store filenames
model_ids_gemma=['/assets/models/google-gemma-3-it-4b']
MAX_NEW_TOKENS = 500 #480 #200

In [50]:
import importlib
import possible_indic_relations as poss_indic_rel
import span_encodings as sp_enc
# Reload the module to reflect changes
importlib.reload(poss_indic_rel)
importlib.reload(sp_enc)
import pandas as pd
pir= poss_indic_rel.possible_relations
pir

ambiguos_words = list(pir.keys())
span_encodings = sp_enc.span_encodings

# drop "child" item from ambiguos_words
index_child = ambiguos_words.index("child")
ambiguos_words.pop(index_child)
print("ambiguos_words", ambiguos_words)

lang_script_list = [
    'hin_Deva', 
    'guj_Gujr',
    'mar_Deva', 
    'ory_Orya',
     'ben_Beng', 
    'tam_Taml', 
    'pan_Guru',
     'tel_Telu',
      'mal_Mlym', 'kan_Knda', 
                           ]

lang_code_map = {
    'eng_Latn': 'Eng',
    'hin_Deva': 'Hin',
    'guj_Gujr': 'Guj',
    'kan_Knda': 'Kan',
    'mal_Mlym': 'Mal',
    'mar_Deva': 'Mar',
    'tam_Taml': 'Tam',
    'tel_Telu': 'Tel',
    'pan_Guru': 'Pun',
    'ben_Beng': 'Ben',
    'ory_Orya': 'Odi'
}


In [11]:
# general_examples = {
#                         0: [  
#                             "ग्राहक दुकानदार से कीमत कम करने के लिए मोलभाव कर रहा है।",
#                             "खरीदार उत्पाद की कीमत पर चर्चा कर रहा है।",
#                             "अभिनेता फिल्म की शूटिंग कर रहा है।",
#                             "गायक नया गीत गा रहा है।",
#                             "खिलाड़ी फुटबॉल खेल रहा है।"
#                         ],
#                         1: [  
#                             "ग्राहिका दुकानदार से कीमत कम करने के लिए मोलभाव कर रही है।",
#                             "खरीदारी करने वाली महिला उत्पाद की कीमत पर चर्चा कर रही है।",
#                             "अभिनेत्री फिल्म की शूटिंग कर रही है।",
#                             "गायिका नया गीत गा रही है।",
#                             "खिलाड़ी फुटबॉल खेल रही है।"
#                         ],
#                         2: [  
#                             "ग्राहक दुकानदार से कीमत कम करने के लिए बातचीत कर रहे हैं।",
#                             "खरीदार उत्पाद की कीमत पर चर्चा कर रहे हैं।",
#                             "कलाकार फिल्म की शूटिंग कर रहे हैं।",
#                             "गायन समूह नया गीत गा रहा है।",
#                             "टीम ने शानदार फुटबॉल खेला।"
#                         ]
#                     }
# fem_examples = {
#     1: ["माँ का प्रेम और करुणा संपूर्ण सृष्टि को ऊर्जा देती है।",
#     "देवी सरस्वती ज्ञान और विद्या की अधिष्ठात्री हैं।",
#     "नारी शक्ति का स्वरूप सृजन और पालन में निहित है।",
#     "योगिनी ध्यान और साधना से आत्मज्ञान प्राप्त करती है।",
#     "गृहलक्ष्मी अपने परिवार के सुख और समृद्धि का आधार होती है।",
#     "कला की साधिका अपनी कल्पनाओं को साकार करती है।",
#     "संगीतकारा अपनी सुरों से वातावरण को मधुर बना देती है।",
#     "रचनात्मक महिला अपने विचारों से समाज को प्रेरित करती है।",
#     "ऋषिका वेदों के गूढ़ ज्ञान को समझने में सक्षम होती है।",
#     "प्रकृति स्वयं मातृशक्ति का प्रतीक है, जो पोषण करती है।",
#     "समुद्र की लहरें देवी की तरह रहस्यमयी और शक्तिशाली होती हैं।",
#     "शक्ति स्वरूपा स्त्री विपरीत परिस्थितियों में भी धैर्य रखती है।",
#     "कवयित्री अपने शब्दों से लोगों की भावनाओं को छू लेती है।",
#     "संवेदनशील नारी दूसरों की पीड़ा को समझने में सक्षम होती है।",
#     "युद्ध में वीरांगना अपने पराक्रम से शत्रुओं का सामना करती है।",
#     "योग साधिका आत्मा और शरीर के संतुलन को बनाए रखती है।",
#     "नृत्यांगना अपने नृत्य से भावनाओं को अभिव्यक्त करती है।",
#     "चिकित्सिका अपने ज्ञान से रोगियों को स्वस्थ करती है।",
#     "शिक्षिका अपने विद्यार्थियों को ज्ञान की रोशनी से आलोकित करती है।",
#     "धार्मिक महिला अपने भक्ति और श्रद्धा से जीवन को संवारती है।"]
# }


In [12]:
general_topics=["occupation", " religion", " sport", " politics", "  health", " finance", " education", " farming", " entertainment", "  news", "  daily conversation", "  weather", " technology", " conflicts", " controversials", 
        " international", "  UN", " travel", " tourism", " shopping", " baby care", " valentines", "  soldiers", " prisioners", " soul actions", " nature", " pollution", " bio hazards", " elders", " family", 
        "  social studies", " maths", " literature", "  physics", " chemistry", "  biology", " Indian history ", " civics", " geography", "  computer", " physical education", "  arts and craft", "  food", 
        " clothes", " water shortage", " road blockage", " traffic", " arriving late", " bargaining", " toys", " games", " deep talks", "declarative", "interrogative", "imperative",  "exclamatory", "safe work space", 
        "medical checkups", "self-reliant", "future India",   "legal", "governance",  
  "STEM" ,   "business", "sports",  "culture", "Alarm","Audio","Calendar / Events", "Communication","DateTime","Email","Finance",  "General","Home Automation", "Location","Music","News","Reminders","Social Media",  
  "Travel","Weather","BookRestaurant","BookFlight","BookHotel","GetDirections","GetPlaceDetails","GetWeather","SearchForInformation","GetNews","PlayMusic","ControlVolume","SetAlarm","CancelAlarm","SetReminder","MakeCall",
  "SendMessage","SendEmail","CheckEmail","GetTime","GetDate","Greeting","Farewell","AskHowAreYou","ThankYou","TellAJoke","TurnOnLights", "Social media interactions",  "Financial transactions"  ]

fem_topics =  ['menstruation', 'pregnancy', 'childbirth', 'breastfeeding', 'menopause', 'ovulation', 'hormones', 'gynecologist', 'obstetrician', 'midwife', 'doula', 'contraception', 'abortion', 'fertility', 'infertility', 'endometriosis', 'polycystic ovary syndrome', 'uterus', 'ovaries', 'marriage', 'divorce', 'single mother', 
               'surrogacy','domestic violence', 'sexual assault', 'rape', 'sexual harassment', 'goddess', 'beauty pageant', 'feminism',]

additional_topics = [ "Housewife", "Policewoman", "CEO", "Nurse", "Teacher", "Chef", "Politician", "Engineer", "Laborer", "Bollywood",  
    "Karwa Chauth", "Raksha Bandhan", "Teej", "Kanyadaan", "Mehendi", "Navratri", "Eid", "Holi", "Diwali", "Chhath Puja",  
    "Cricket", "Kabaddi", "Wrestling", "Badminton", "Olympics", "IPL", "Marathon", "Yoga", "Coaching", "School Sports",  
    "Voting", "Protests", "Gender Crime", "Quotas", "News Reporting", "Women’s Rights", "Men’s Rights", "Dowry Law", "Parliament", "Governance",  
   "Maternity", "Paternity", "Skin Care", "Hairfall", "Dieting", "Medical Checkups", "Parlor", "Gym", "Pregnancy", "Breastfeeding",  
    "Budget", "Loans", "Savings", "Shopping", "Investments", "Gold", "Startups", "Stock Market", "Banking", "Credit Cards",  
    "STEM", "IIT", "Nursing", "Fashion", "Leadership", "Arts", "Research", "Admissions", "Govt Jobs", "Engineering",  
    "Agriculture", "Water Shortage", "Pollution", "Bio Hazards", "Organic Farming", "Green Energy", "Monsoon", "Drought", "Farmers’ Protests", "Pesticides",  
    "Serials", "Reality Shows", "Action Films", "Singers", "Web Series", "Pay Gap", "Directors", "Influencers", "Romance", "Bollywood",  
    "Social Media", "Financial Transactions", "Fake News", "News Anchors", "Political Debates", "Public Speaking", "Cyber Safety", "E-Governance", "WhatsApp Forwards", "YouTube Trends",  
    "Solo Travel", "Taxi Safety", "Bike Riders", "Public Transport", "Driving", "Holiday", "Blogging", "Late-Night Travel", "Destinations", "Work-from-Home",  
    "Marriage", "Dowry", "Motherhood", "Fatherhood", "Widowhood", "Siblings", "Bride", "Groom", "Joint Family", "Daughter-Son",  
    "New Year", "Makar Sankranti", "Baisakhi", "Ganesh Chaturthi", "Pongal", "Onam", "Eid-ul-Fitr", "Christmas", "Buddha Purnima", "Muharram",  
    "Cooking", "Shopping", "Cleaning", "Driving", "Bills", "Decor", "Childcare", "Grocery", "Hobbies", "Festivals",  
    "Citizenship", "Elections", "Protests", "RTI", "Lawsuits", "Corruption", "Taxation", "Parliament Sessions", "Women’s Rights", "Legal Awareness"
]
# additional_topics = additional_topics - fem_topics - general_topics
additional_topics = list(set(additional_topics) - set(fem_topics) - set(general_topics))
print(len(additional_topics), len(fem_topics), len(general_topics))

topics=general_topics #[-3:]
# topics=additional_topics
# topics = None # set below
print("topics", len(topics))

In [13]:
# if PROMPT_TYPE == 'female':
#     examples = fem_examples
#     topics = fem_topics
# else:
#     # topics = general_topics
#     topics = additional_topics
#     examples = general_examples

# print(len(topics))

In [14]:
# examples = '''[
#     ..   "पूछती हूँ", "बनाती हूँ", "बचाती हूँ", "डरती हूँ", "हँसती हूँ", "रोती हूँ", "लड़ती हूँ",...
#     "गिरती हूँ", "उठाती हूँ", "सुलाती हूँ", "दिखाती हूँ", "सुनाती हूँ"...
# ]'''

In [15]:
examples_coll ={
    'hin_Deva': {
    "चाची": [
        "चाची ने संपत्ति विवाद को अदालत में उठाया।",
        "मेरी चाची ने वकील से सलाह ली।",
        # "चाची ने ज़मीन के काग़ज़ात की जाँच करवाई।",
        # "मेरी चाची ने कानूनी अधिकारों के लिए याचिका दायर की।",
        # "चाची अदालत में अपने पक्ष को मजबूती से रख रही थीं।",
        # "मेरी चाची ने विरासत से जुड़ा केस जीता।",
        # "चाची ने कानून के तहत अपने हिस्से की माँग की।"
    ],
    "बुआ": [
        "बुआ ने तलाक़ की प्रक्रिया शुरू कर दी है।",
        "मेरी बुआ ने धोखाधड़ी के खिलाफ एफआईआर दर्ज करवाई।",
        # "बुआ कोर्ट में अपने बच्चों की कस्टडी के लिए लड़ रही हैं।",
        # "बुआ ने अपने वकील के साथ कानूनी रणनीति बनाई।",
        # "मेरी बुआ ने कोर्ट में बयान दर्ज कराया।",
        # "बुआ ने न्यायालय से सुरक्षा की माँग की।",
        # "बुआ को संपत्ति विवाद में न्याय मिला।"
    ],
    "ताई": [
        "ताई ने अदालत में दस्तावेज़ पेश किए।",
        "मेरी ताई ने सरकारी वकील से बात की।",
        # "ताई ने अदालत में गवाही दी।",
        # "ताई ने ज़मीन के बंटवारे को लेकर केस दर्ज करवाया।",
        # "मेरी ताई ने अपनी विरासत के अधिकार की कानूनी लड़ाई जीती।",
        # "ताई ने समाजिक कल्याण योजनाओं में धोखाधड़ी की शिकायत की।",
        # "ताई कोर्ट में अपने पक्ष को स्पष्ट कर रही थीं।"
    ],
    "मामी": [
        "मामी ने धोखाधड़ी के मामले में शिकायत दर्ज की।",
        "मेरी मामी ने कानून के तहत गुज़र-बसर भत्ता माँगा।",
        # "मामी ने महिला आयोग में मामला दर्ज कराया।",
        # "मामी अदालत से न्याय की उम्मीद कर रही हैं।",
        # "मामी ने दस्तावेज़ों की वैधता को चुनौती दी।",
        # "मेरी मामी ने कानून सलाहकार से परामर्श लिया।",
        # "मामी ने पुलिस में विधिवत बयान दर्ज करवाया।"
    ],
    "मासी": [
        "मासी ने कानूनी सहायता के लिए अर्जी दी।",
        "मेरी मासी ने अदालत में अपने हक़ की लड़ाई लड़ी।",
        # "मासी ने कोर्ट में संपत्ति पर दावा किया।",
        # "मासी ने गवाहों की सूची वकील को सौंपी।",
        # "मेरी मासी को वकील ने कड़ी दलील देने की सलाह दी।",
        # "मासी ने पुलिस थाने में रिपोर्ट दर्ज करवाई।",
        # "मासी ने पारिवारिक विवाद में अदालत का रुख किया।"
    ]
},
  'guj_Gujr':  {
    "કાકી": [
        "કાકીએ મિલકતના વિવાદ માટે કોર્ટમાં કેસ દાખલ કર્યો.",
        "મારી કાકીએ વકીલની સલાહ લીધી અને કાયદેસર પગલાં લીધા."
    ],
    "ફોઈ": [
        "ફોઈએ કૌટુંબિક હિંસાના કેસમાં ફરિયાદ નોંધાવી.",
        "મારી ફોઈએ કોર્ટમાં પોતાના હક્ક માટે સાક્ષ આપ્યો."
    ],
    "મોટી મા": [
        "મોટી માએ વારસાની હક્ક માટે કાયદેસર કાર્યવાહી કરી.",
        "મોટી માંએ કોર્ટમાં કાગળો રજૂ કર્યા."
    ],
    "મામી": [
        "મામીએ સ્ત્રી અધિકાર માટે ફરિયાદ નોંધાવી.",
        "મારી મામીએ પોલીસ સ્ટેશનમાં લેખિત ફરિયાદ આપી."
    ],
    "મોશી": [
        "મોશીએ કાયદેસર હક્ક માટે અરજદારી કરી.",
        "મોશી કોર્ટના દરવાજા સુધી પહોંચ્યા ન્યાય માટે."
    ]
}, 'ory_Orya': {

    "ଖୁଡି": [
        "ଖୁଡି ଜମି ସମ୍ପର୍କିତ ମାମଲା ପାଇଁ ଆଇନଜୀବୀ ସହ ସମ୍ପର୍କ କଲେ।",
        "ମୋ ଖୁଡି ନିଜର ଅଧିକାର ପାଇଁ ଆଇନ ମାର୍ଗରେ ଯିବାକୁ ଚାହାଁଛନ୍ତି।"
    ],
    "ପିସି": [
        "ପିସି ନ୍ୟାୟାଳୟ ରେ ଗवाही ଦେଲେ।",
        "ମୋ ପିସି ଏକ ଆଇନ ମାମଲା ରେ ଜିତିଲେ।"
    ],
    "ବଡ ମା": [
        "ବଡ ମା ଏକ ଥାନାରେ ରିପୋର୍ଟ ଦାଖଲ କରିଥିଲେ।",
        "ମୋ ବଡ ମା ଜମି ବିବାଦ ପାଇଁ ଅଦାଲତ କୁ ଗଲେ।"
    ],
    "ମାଏନ": [
        "ମାଏନ ନିଜ ଆଇନିକ ଅଧିକାର ନିମନ୍ତେ ଆବେଦନ କଲେ।",
        "ମୋ ମାଏନ ନ୍ୟାୟ ଲାଗି ମହିଳା କମିଶନ୍ ରେ ଅଭିଯୋଗ କଲେ।"
    ],
    "ମାଉସି": [
        "ମାଉସି ଜନ୍ମସ୍ଥଳୀର ଥାନାରେ ଏକ ଅଭିଯୋଗ ଦାଖଲ କଲେ।",
        "ମୋ ମାଉସି ଆଇନ ଅନୁଯାୟୀ ନ୍ୟାୟ ପାଇଁ ଚେଷ୍ଟା କରୁଛନ୍ତି।"
    ]
}
,

  'mal_Mlym': {
    "അമ്മായിയമ്മ": [
        "അമ്മായിയമ്മ ഭൂമി തർക്കത്തിൽ കോടതിയെ സമീപിച്ചു.",
        "എന്റെ അമ്മായിയമ്മ നിയമോപദേശം തേടി രംഗത്തെത്തി."
    ],
    "പിഷി": [
        "പിഷി കുടുംബപരമായ പ്രശ്നങ്ങൾക്കായി കേസ് കൊടുത്തു.",
        "എന്റെ പിഷി കോടതിയിൽ സാക്ഷ്യം പറഞ്ഞു."
    ],
    "വലിയമ്മ": [
        "വലിയമ്മ അവകാശങ്ങൾക്കായി നിയമപരമായി നടപടി ആരംഭിച്ചു.",
        "എന്റെ വലിയമ്മ വക്കീലുമായി നിയമപ്രവർത്തനങ്ങൾ തുടങ്ങി."
    ],
    "മാമി": [
        "മാമി സ്ത്രീകളും കുട്ടികളും സംരക്ഷണത്തിനായി പരാതിപ്പെട്ടു.",
        "എന്റെ മാമി പൊലീസ് സ്റ്റേഷനിൽ പരാതി നൽകി."
    ],
    "മാമ്മ": [
        "മാമ്മ കോടതിയിൽ തർക്കം പരിഹരിക്കാൻ ശ്രമിച്ചു.",
        "എന്റെ മാമ്മ നിയമം അനുസരിച്ച് അധികാരങ്ങൾ ആവശ്യപ്പെട്ടു."
    ]
  },
  'kan_Knda': {
    "ಅತ್ತೆ": [
        "ಅತ್ತೆ ಜಮೀನು ವಿವಾದದ ಕುರಿತು ನ್ಯಾಯಾಲಯದಲ್ಲಿ ಅರ್ಜಿ ಸಲ್ಲಿಸಿದರು.",
        "ನನ್ನ ಅತ್ತೆ ಕಾನೂನು ಸಲಹೆಗಾಗಿ ವಕೀಲರನ್ನು ಭೇಟಿಯಾದರು."
    ],
    "ಪಿಸಿಯವರು": [
        "ಪಿಸಿಯವರು ಕೌಟುಂಬಿಕ ಸಮಸ್ಯೆಗೆ ಸಂಬಂಧಿಸಿದಂತೆ ತಾಕೀತು ಪತ್ರ ನೀಡಿದರು.",
        "ನನ್ನ ಪಿಸಿಯವರು ನ್ಯಾಯಾಲಯದಲ್ಲಿ ತಮ್ಮ ಹಕ್ಕುಗಳನ್ನು ಬೇಡಿಕೆ ಹಾಕಿದರು."
    ],
    "ದೊಡ್ಡ ಅಮ್ಮ": [
        "ದೊಡ್ಡ ಅಮ್ಮ ಕಾನೂನು ಪ್ರಕ್ರಿಯೆ ಬಗ್ಗೆ ಸೂಕ್ತ ದಾಖಲೆಗಳನ್ನು ಸಂಗ್ರಹಿಸಿದರು.",
        "ನನ್ನ ದೊಡ್ಡ ಅಮ್ಮ ನ್ಯಾಯಾಲಯದಲ್ಲಿ ಸಾಕ್ಷ್ಯಾಧಾರ ನೀಡಿದರು."
    ],
    "ಮಾಮಿ": [
        "ಮಾಮಿ ಮಹಿಳಾ ಕಮಿಷನ್‌ಗೆ ದೂರು ನೀಡಿದರು.",
        "ನನ್ನ ಮಾಮಿ ಕಾನೂನು ಮೂಲಕ ನ್ಯಾಯವನ್ನು ಪಡೆಯಲು ಯತ್ನಿಸುತ್ತಿದ್ದಾರೆ."
    ],
    "ಚಿಕ್ಕಮ್ಮ": [
        "ಚಿಕ್ಕಮ್ಮನು ಹಕ್ಕಿಗಾಗಿ ನ್ಯಾಯಾಲಯದಲ್ಲಿ ಅರ್ಜಿ ಸಲ್ಲಿಸಿದರು.",
        "ನನ್ನ ಚಿಕ್ಕಮ್ಮ ಕಾನೂನು ಪ್ರಕ್ರಿಯೆಯಲ್ಲಿ ಸಕ್ರಿಯವಾಗಿ ಭಾಗವಹಿಸುತ್ತಿದ್ದಾರೆ."
    ]
  },
  'mar_Deva': {
    "काकू": [
        "काकूंनी मालमत्तेच्या वादासाठी वकिलांची मदत घेतली.",
        "माझ्या काकूंनी न्यायालयात आपला हक्क सिद्ध केला."
    ],
    "पिसी": [
        "पिसीने कौटुंबिक हिंसाचारविरोधात तक्रार दाखल केली.",
        "माझ्या पिसीने कोर्टात साक्ष दिली."
    ],
    "मोठी आई": [
        "मोठ्या आईने जमिनीच्या हक्कासाठी केस दाखल केला.",
        "माझ्या मोठ्या आईने वकिलांच्या मार्गदर्शनाने कायदेशीर कारवाई केली."
    ],
    "मावस": [
        "मावसने पोलिसांत वारसाहक्काची तक्रार नोंदवली.",
        "माझ्या मावसने कोर्टात अर्ज दाखल केला."
    ],
    "मावशी": [
        "मावशीने कायदेशीर सल्ला घेऊन निर्णय घेतला.",
        "माझ्या मावशीने महिला आयोगाकडे आपली तक्रार मांडली."
    ]
  },
  'tam_Taml': {
    "அத்தை": [
        "அத்தை நீதிமன்றத்தில் சொத்து உரிமைக்காக வழக்கு தொடர்ந்தார்.",
        "என் அத்தை சட்ட ஆலோசனை பெற்ற பிறகு நடவடிக்கை எடுத்தார்."
    ],
    "பாட்டி அத்தை": [
        "பாட்டி அத்தை வீட்டுவசதி உரிமை குறித்து புகார் செய்தார்.",
        "என் பாட்டி அத்தை நீதிமன்றத்தில் சாட்சியாக வெளியேறினார்."
    ],
    "பெரியம்மா": [
        "பெரியம்மா வக்கீலின் உதவியுடன் வழக்கு தாக்கல் செய்தார்.",
        "என் பெரியம்மா சட்டப்படி தனது உரிமையை கோரினார்."
    ],
    "மாமி": [
        "மாமி குடும்ப வன்முறையுக்கு எதிராக போலீசில் புகார் செய்தார்.",
        "என் மாமி சட்ட மையத்தில் மனு தாக்கல் செய்தார்."
    ],
    "மாமி (2)": [
        "மாமி நீதிமன்றத்தில் தனது நில உரிமையை நிரூபித்தார்.",
        "என் மாமி சட்ட ஆலோசகரை சந்தித்தார்."
    ]
  },
  'tel_Telu': {
    "అత్త": [
        "అత్త గారు ఆస్తి హక్కుల కోసం కోర్టులో కేసు వేశారు.",
        "నా అత్త గారు న్యాయ సహాయానికి న్యాయవాదిని సంప్రదించారు."
    ],
    "పిన్నమ్మ": [
        "పిన్నమ్మ గారు పోలీసులకు ఫిర్యాదు చేశారు.",
        "పిన్నమ్మ కోర్టులో తగిన ఆధారాలతో హాజరయ్యారు."
    ],
    "పెద్దమ్మ": [
        "పెద్దమ్మ తన హక్కుల కోసం న్యాయపరమైన చర్యలు చేపట్టారు.",
        "నా పెద్దమ్మ ఆస్తి సంబంధిత పత్రాలు కోర్టులో సమర్పించారు."
    ],
    "మామి": [
        "మామి మహిళా హక్కుల కోసం లీగల్ నోటీసు ఇచ్చారు.",
        "మా మామి న్యాయంగా తమ హక్కును పొందేందుకు కోర్టును ఆశ్రయించారు."
    ],
    "మావెము": [
        "మావెము న్యాయ సలహా తీసుకొని ఫిర్యాదు చేశారు.",
        "మావెము పోలీసు స్టేషన్‌లో పిర్యాదు చేశారు."
    ]
  },
  'pan_Guru': {
    "ਚਾਚੀ": [
        "ਚਾਚੀ ਨੇ ਜਾਇਦਾਦ ਦੇ ਮੁੱਦੇ 'ਤੇ ਅਦਾਲਤ 'ਚ ਮਾਮਲਾ ਚਲਾਇਆ।",
        "ਚਾਚੀ ਨੇ ਆਪਣੇ ਕਾਨੂੰਨੀ ਹੱਕ ਲਈ ਵਕੀਲ ਨਾਲ ਸਲਾਹ ਲਈ।"
    ],
    "ਬੁਆ": [
        "ਬੁਆ ਨੇ ਘਰੇਲੂ ਹਿੰਸਾ ਖਿਲਾਫ ਕੇਸ ਦਰਜ ਕਰਵਾਇਆ।",
        "ਬੁਆ ਅਦਾਲਤ ਵਿੱਚ ਆਪਣੇ ਹੱਕ ਦੀ ਗਵਾਹੀ ਦੇ ਰਹੀ ਸੀ।"
    ],
    "ਵੱਡੀ ਮਾਂ": [
        "ਵੱਡੀ ਮਾਂ ਨੇ ਵਾਰਸਾਨਾ ਹੱਕ ਲਈ ਕਾਨੂੰਨੀ ਕਾਰਵਾਈ ਸ਼ੁਰੂ ਕੀਤੀ।",
        "ਵੱਡੀ ਮਾਂ ਨੇ ਜਾਇਦਾਦ ਸੰਬੰਧੀ ਦਸਤਾਵੇਜ਼ ਅਦਾਲਤ ਵਿਚ ਪੇਸ਼ ਕੀਤੇ।"
    ],
    "ਮਾਮੀ": [
        "ਮਾਮੀ ਨੇ ਔਰਤਾਂ ਦੇ ਹੱਕ ਲਈ ਸ਼ਿਕਾਇਤ ਕੀਤੀ।",
        "ਮਾਮੀ ਨੇ ਪੁਲਿਸ ਨੂੰ ਲਿਖਤੀ ਸ਼ਿਕਾਇਤ ਦਿੱਤੀ।"
    ],
    "ਮੌਸੀ": [
        "ਮੌਸੀ ਨੇ ਆਪਣੇ ਕਾਨੂੰਨੀ ਹੱਕ ਲਈ ਅਰਜ਼ੀ ਦਿੱਤੀ।",
        "ਮੌਸੀ ਨੇ ਅਦਾਲਤ ਤੋਂ ਇਨਸਾਫ਼ ਦੀ ਮੰਗ ਕੀਤੀ।"
    ]
  },
  'ben_Beng': {
    "কাকী": [
        "কাকী সম্পত্তি সংক্রান্ত বিষয়ে মামলা দায়ের করেছেন।",
        "আমার কাকী আইনজীবীর সঙ্গে পরামর্শ করেছেন।"
    ],
    "পিসি": [
        "পিসি আদালতে নিজের অধিকারের জন্য দাঁড়িয়েছিলেন।",
        "আমার পিসি একটি পারিবারিক মামলায় জয়লাভ করেছেন।"
    ],
    "বড় মা": [
        "বড় মা উত্তরাধিকার সংক্রান্ত মামলা আদালতে তুলেছেন।",
        "আমার বড় মা কোর্টে প্রয়োজনীয় কাগজ জমা দিয়েছেন।"
    ],
    "মামি": [
        "মামি নারী হিংসার বিরুদ্ধে অভিযোগ দায়ের করেছেন।",
        "আমার মামি আইনের সাহায্যে ন্যায় বিচারের জন্য লড়ছেন।"
    ],
    "মাসি": [
        "মাসি সম্পত্তির ভাগ নিয়ে আদালতে গিয়েছেন।",
        "আমার মাসি পুলিশ স্টেশনে লিখিত অভিযোগ দাখিল করেছেন।"
    ]
  }
}
for lang in examples_coll.keys():
    val = examples_coll[lang]
    for k in val.keys():
        v = val[k]
        examples_coll[lang][k] = str(v)+'...'



print(len(examples_coll))
print(examples_coll['ory_Orya'].keys())
print(examples_coll['ory_Orya'])


#### Functions and Utilities

In [38]:


def model_init(model_path):
    tokenizer = transformers.AutoTokenizer.from_pretrained(model_path)
    print("tokenizer loaded")

    model = Gemma3ForConditionalGeneration.from_pretrained(
        # model_id, device_map="auto"
        model_path, device_map='auto'
    ).eval()

    processor = AutoProcessor.from_pretrained(model_path)


    return tokenizer, model, processor

In [39]:
def get_now_str():
    now_str = dt.datetime.now().strftime(datetime_format)
    # print(now_str)
    return now_str

def save_logs(now_str, messages=None, inputs=None, decoded=None, time_diff=None, tgt_lang_code=None):
    with open('response_logs.txt', 'a+') as f:
        # write decoded to file with timestamp now_str
        f.write("=========================================\n")
        f.write(f"\nTime Stamp: {now_str} :: Time Diff: {time_diff}  :: TGT_LANG:: {tgt_lang_code}\n Messages: {messages} \nInputs: {inputs} \nOutput: {decoded}\n")
        f.write("=========================================\n")
        f.close()
    # print("Response saved to file")

def sav_outputs(now_str, outputs, tgt_lang_code):
    with open(tgt_lang_code + '_outputs_logs.txt', 'a+') as f:
        # write decoded to file with timestamp now_str
        f.write("=========================================\n")
        f.write(f"\nTime Stamp: {now_str} :: TGT_LANG:: {tgt_lang_code} \nOutput: **<<{str(outputs)}>>**\n")
        f.write("=========================================\n")
        f.close()
    # print("Outputs saved to file")


In [57]:
# processor = AutoProcessor.from_pretrained(model_id)

def get_message(tgt_lang = 'Hindi', examples=None, num=7, topic='law', prompt=None, mapping=None):
        
        prompt = '''Generate total ''' +str((len(mapping)*num)) +" "+ tgt_lang+''' sentences and return a python dictionary where  keys are ''' + str(mapping) + '''
and each of the corresponding values are the list of ''' +str((num)) +" "+ tgt_lang+''' sentences corresponding to using each relation and using the topic: '''+ topic +'''. Generate gramatically correct sentences where each given relation is used respectively.
Do not associate with any bias or stereotypes associated with subject, relation and gender. No need of comments or explanations or English texts. Make the sentences as varied as possible. While generating relations in ''' +tgt_lang+''' sentences, do not use neutral terms.
Generate the sentences for the above mentioned relations and not exactly same as the given examples.'''
              

        messages = [
            {
                "role": "system",
                "content": [{"type": "text", "text": "You are a helpful assistant that understands "+tgt_lang+" and can generate sentences in "+tgt_lang+"."}]
            },
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt+'''
                    Sample Output:
                    sentences = ''' +str(examples)
        }
                ]
            }
        ]
        return messages


def get_prompt_inputs(processor, messages, model, add_gen_prompt=True, tokenize=True, return_dict=True, return_tensors="pt"):
    inputs = processor.apply_chat_template(
        messages, add_generation_prompt=add_gen_prompt, tokenize=tokenize,
        return_dict=return_dict, return_tensors=return_tensors,
    ).to(model.device, dtype=torch.bfloat16)
    return inputs


def get_decoded_response(model, processor, inputs, max_new_tokens=MAX_NEW_TOKENS, do_sample=False, temperature=1.5, num_return_sequences=1):
    input_len = inputs["input_ids"].shape[-1]

    with torch.inference_mode():
        generation = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=do_sample, 
                                    temperature=temperature, num_return_sequences=num_return_sequences, 
                                    max_length=input_len + max_new_tokens)
        generation = generation[0][input_len:]

    decoded = processor.decode(generation, skip_special_tokens=True)
    # print(decoded)
    return decoded

# **Overall Impression:** The image is a close-up shot of a vibrant garden scene, 
# focusing on a cluster of pink cosmos flowers and a busy bumblebee. 
# It has a slightly soft, natural feel, likely captured in daylight.


In [58]:
def get_extracted_text(text):
    start_ind = text.find('```python')
    end_ind = text.find('```', start_ind+1)
    # if end_idx not found then last index of text
    if end_ind == -1:
        end_ind = len(text)
    extracted_text = text[start_ind:end_ind]
    return extracted_text


In [20]:
pir['grandfather']['ory_Orya'].keys()

In [21]:
# topics

To generate as per topics

In [ ]:
for model_path in model_ids_gemma:
    tokenizer, model, processor = model_init(model_path)
    for word in tqdm.tqdm(ambiguos_words):
        for topic in tqdm.tqdm(topics):
            for tgt_lang in tqdm.tqdm(lang_script_list):
                print("Word: ", word, "Topic: ", topic, "TGT_LANG: ", tgt_lang)
                now_str = get_now_str()
                start_time = dt.datetime.strptime(now_str, datetime_format)
                save_logs(now_str, 'Model: '+model_path + '\nword: '+ word+ '\nTopic: '+topic + '\nTGT_LANG: '+tgt_lang + '\nTASK : Generating data for relations', tgt_lang_code=tgt_lang)
                # messages = get_message(examples= examples, topic=topic)
                messages = get_message(tgt_lang=tgt_lang, examples= examples_coll[tgt_lang], topic=topic, num =3,   mapping= list(pir[word][tgt_lang].keys()) )
                inputs = get_prompt_inputs(processor, messages, model)
                now_str2 = get_now_str()
                end_time = dt.datetime.strptime(now_str2, datetime_format)
                save_logs(now_str2, messages, inputs=inputs['input_ids'], time_diff=  end_time-start_time, tgt_lang_code=tgt_lang)
                start_time = end_time
                decoded = get_decoded_response(model, processor, inputs)
                now_str3 = get_now_str()
                end_time = dt.datetime.strptime(now_str3, datetime_format)
                save_logs(now_str3, decoded=decoded, time_diff= end_time-start_time, tgt_lang_code=tgt_lang)
                extracted_txt = get_extracted_text(decoded)
                now_str = get_now_str()
                sav_outputs(now_str, extracted_txt, tgt_lang)

In [24]:
word

# indic_sent_dict Analysis

In [226]:
indic_sent_dict  

In [ ]:
import csv

# Sample dictionary structure
# Convert the dictionary to a list of rows (one row per sentence)
rows = []
for word, lang_dict in indic_sent_dict.items():
    for lang, term_dict in lang_dict.items():
        for indic_term, sentences in term_dict.items():
            for sentence in sentences:
                rows.append({
                    'word': word,
                    'lang': lang,
                    'indic_term': indic_term,
                    'sentence': sentence
                })

# Write to CSV
with open('output.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['word', 'lang', 'indic_term', 'sentence']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    writer.writerows(rows)

print("CSV file 'output.csv' created successfully.")


In [230]:
def get_message_eng(sents=None, rootword=None, tgt_lang = 'Hindi', examples=None):
        prompt = '''Return the python list of English sentences using the word "'''+rootword+'''" as the translation of following sentences in ''' +tgt_lang+ '\n'+ "\n- ".join(sents)+'''
No need to write explaination and do not specify maternal or paternal. Simply provide Python list containing '''+ str(len(sents)) +'''  English sentences as shown below. '''

                  

        messages = [
            {
                "role": "system",
                "content": [{"type": "text", "text": "You are a helpful assistant that correctly translates from "+tgt_lang+" to English."}]
            },
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt+'''
                    Sample Python Code Output:\sentences = ''' +(str(examples[:len(sents)]) if examples else str(examples))
        }
                ]
            }
        ]
        return messages

In [228]:
df_translations = pd.DataFrame(columns=['word', 'eng_text','tgt_lang_text', 'tgt_lang', 'indic_text'])

In [229]:
for model_path in model_ids_gemma:
    tokenizer, model, processor = model_init(model_path)

batch_size = 10

In [223]:
now_str = get_now_str()
now_str

In [273]:
def preprocess_text(code):
    # print("code:", code)
    # Remove any unwanted characters or formatting
    arr = []
    # deep copy code to a temp variable
    code_temp = code
    code = code.strip()
    code = code.replace("...", "")
    search_str = '```python\nsentences = '
    start_idx = code.find(search_str)
    end_idx = code.find(']', start_idx+1)
    code = code[start_idx+len(search_str):]
    # print("end_idx of ]:", end_idx)
    if end_idx == -1 :
        # print("code this:", code)
        code = code.strip()
        if not code.endswith(']'):
            code = code.strip()
            if code.endswith(".'") or code.endswith('."'):
                code = code + "]"
            elif code.endswith(" '") or code.endswith(' "'):
                code = code[:-1] + "]"
            else: 
                # Try to find the last incomplete string
                match = re.findall(r'([\'"])[^\'"]*$' , code)
                quote_type = match[-1] if match else '"'  # fallback
                # print("quote_type:", quote_type, "code:", code)
                # Step 1: Add missing closing quote if needed
                if code.count(quote_type) % 2 != 0:
                    code += quote_type
                # Step 2: Add missing closing bracket if needed
                if not code.endswith("]"):
                    code = code  + "]"
    else:
        code = code[: end_idx+1]
            
    try:
        #  dict_code = eval(code
        arr = ast.literal_eval(code)
    except Exception as e:
        print("\n\n! ! ! eval error code:", code, "\n with error : ", e, '\n code_temp', code_temp)

    return arr

stri= '''```python
sentences = [
    "My grandmother told me stories.",
    "My grandmother is a skilled weaver.",
    "My grandmother told me stories about life.",
    "My grandmother taught me how to make traditional dishes.",
    "My grandmother told me stories about life.",
    "My grandmother told me stories, which included struggles in life.",
    "My grandm '''

preprocess_text(stri)

In [283]:
batch_size=2

In [279]:
indic_sent_dict['grandfather']['hin_Deva']

In [7]:
try:
    for word in indic_sent_dict:
        print(">> Word: ", word)   
        for lang_code in indic_sent_dict[word]:
            print(">>>> TGT_LANG: ", lang_code)
            df_lang_word=None
            df_lang_word = pd.DataFrame(columns=['word', 'eng_text','tgt_lang_text', 'tgt_lang', 'indic_text'])
            transl_arr =[]
            for key in indic_sent_dict[word][lang_code]:
                print(">>>>>> Key: ", key)
                indic_sents = indic_sent_dict[word][lang_code][key]
                # indic_sents = indic_sents[:45]
                l = len(indic_sents)
                l = l if l < 5 else 5
                indic_sents = indic_sents[:l]
                for i in tqdm.tqdm(range(0, len(indic_sents), batch_size)):
                    batch = indic_sents[i:i+batch_size]

                    sents = batch
                    now_str1 = get_now_str()        
                    start_time = dt.datetime.strptime(now_str1, datetime_format)
                    save_logs(now_str1, messages= 'Model: '+model_path + '\n Task: English equivalents; i: '+ str(i )+ '\nBatch Size: '+  str(batch_size))
                    # messages = get_message(examples= examples, topic=topic)
                    messages = get_message_eng(tgt_lang=lang_code_map[lang_code], rootword=word, sents=sents)
                    inputs = get_prompt_inputs(processor, messages, model)
                    now_str2 = get_now_str()
                    end_time = dt.datetime.strptime(now_str2, datetime_format)
                    save_logs(now_str2, messages, inputs, time_diff=  end_time-start_time)
                    start_time = end_time
                    decoded = get_decoded_response(model, processor, inputs, max_new_tokens=100)
                    now_str3 = get_now_str()
                    end_time = dt.datetime.strptime(now_str3, datetime_format)
                    save_logs(now_str3, decoded=decoded, time_diff= end_time-start_time, tgt_lang_code=lang_code)
                    extracted_txt = get_extracted_text(decoded)
                    # print("Extracted Text: ", extracted_txt, type(extracted_txt))
                    now_str4 = get_now_str()
                    sav_outputs(now_str4, extracted_txt, lang_code)
                    eng_sents = preprocess_text(extracted_txt)
                    print("English Sentences length: ", len(eng_sents), "sents length: ", len(sents))
                    temp_arr = []
                    for _ in range(len(eng_sents)):
                        temp_arr.append([word, eng_sents[_], sents[_], lang_code, key])
                    # break
                    # if i > 1* batch_size:
                    #     break
                print("Batch processed: ", i, "size : ", len(temp_arr), "size of transl_arr: ", len(transl_arr))
                transl_arr.extend(temp_arr)
                print("Total translations so far: ", len(transl_arr), transl_arr)   
                temp_arr=[]
                # df_translations = pd.concat([df_translations, pd.DataFrame(transl_arr, columns=['word', 'eng_text','tgt_lang_text', 'tgt_lang', 'indic_text'])], ignore_index=True
                # )
                # df_translations.to_csv('translations'+now_str4+'.csv', index=False)
            print(len(transl_arr), df_lang_word.shape)
            df_lang_word = pd.concat([df_lang_word, pd.DataFrame(transl_arr, columns=['word', 'eng_text','tgt_lang_text', 'tgt_lang', 'indic_text'])], ignore_index=True
            )
            df_lang_word.to_csv('translations_'+word+'_'+lang_code+now_str4+'.csv', index=False)
            # break
except Exception as e:
    print("!!! Error: ", e)
    print("Word: ", word, "TGT_LANG: ", lang_code, "i: ", i)
    print("Batch Size: ", batch_size, '\n sents: ', sents)
    print("Decoded: ", decoded)
    print("Extracted Text: ", extracted_txt)
    # print("Inputs: ", inputs['input_ids'])
    print("Messages: ", messages)
    print("Error in processing the word and language")


In [ ]:
temp_arr = []
for _ in range(len(eng_sents)):
    temp_arr.append([word, eng_sents[_], sents[_], lang_code, key])
    # break
    # if i > 1* batch_size:
    #     break
transl_arr.extend(temp_arr)
                # temp_arr=[]
                # df_translations = pd.concat([df_translations, pd.DataFrame(transl_arr, columns=['word', 'eng_text','tgt_lang_text', 'tgt_lang', 'indic_text'])], ignore_index=True
                # )
                # df_translations.to_csv('translations'+now_str4+'.csv', index=False)
print(len(transl_arr), df_lang_word.shape)
            

In [ ]:
 df_lang_word = pd.concat([df_lang_word, pd.DataFrame(transl_arr, columns=['word', 'eng_text','tgt_lang_text', 'tgt_lang', 'indic_text'])], ignore_index=True
            )
            df_lang_word.to_csv('translations_'+word+'_'+lang_code+now_str4+'.csv', index=False)
           

In [267]:
df_lang_word

In [ ]:
break runn all:)


### To generate from prompt list

In [ ]:
def get_effective_text(text, combo):
    msg ={}
    try:
        search_str = '```python\nsentences = '
        start_ind = text.find(search_str)
        end_ind = text.find('```', start_ind+1)
        # if end_idx not found then last index of text
        if end_ind == -1:
            end_ind = len(text)
        extracted_text = text[start_ind+len(search_str):end_ind]
        extracted_text = extracted_text.replace('\n', '')

        # if extracted text ends with ] then take whole text else, take till last occurence '",'
        if not extracted_text.endswith(']'):
            last_ind = extracted_text.rfind('",')
            if last_ind != -1:
                extracted_text = extracted_text[:last_ind+1] + ']'
            else:
                print("Invalid list:: for text", text, " ; extracted_txt: ", extracted_text)
                extracted_text = extracted_text + ']'

        if 'male' in combo:
            msg[0] =    ast.literal_eval(extracted_text)
        elif 'female' in combo:
            msg[1] = ast.literal_eval(extracted_text)   
        else:
            msg[2] = ast.literal_eval(extracted_text)
    except Exception as e:
        print("Error in extracting text: ", e)
        save_logs("Error in extracting text: ", e, text) 
    return str(msg)     


In [ ]:
from itertools import product
import random

# Define the dimensions
politeness_forms = ['polite', 'non-polite']
genders = ['male', 'female', 'they']
persons = ['first', 'second', 'third']
tenses = [
    'present simple', 'present continuous', 'present perfect', 'present perfect continuous',
    'past simple', 'past continuous', 'past perfect', 'past perfect continuous',
    'future'
]

# Generate all 162 combinations
# combinations = list(product(politeness_forms, genders, persons, tenses))
tenses = [
    'present simple', 'present continuous', 'present perfect', 'present perfect continuous',
    'past simple', 'past continuous', 'past perfect', 'past perfect continuous',
    'future'
]

# First person: polite doesn't matter; gender matters only in past tenses
first_person_combinations = []
for tense in tenses:
    if 'past' in tense:
        for gender in ['male', 'female']:
            first_person_combinations.append(('first', gender, 'neutral', tense))
    else:
        first_person_combinations.append(('first', 'neutral', 'neutral', tense))

# Second person: tu, tum, aap = different politeness forms
# Gender matters in past tenses
second_person_combinations = []
for politeness in ['tu', 'tum', 'aap']:
    for tense in tenses:
        if 'past' in tense:
            for gender in ['male', 'female']:
                second_person_combinations.append(('second', gender, politeness, tense))
        else:
            second_person_combinations.append(('second', 'neutral', politeness, tense))

# Third person:
# Singular (male/female): gender matters
# Plural (they): gender-neutral
third_person_combinations = []
for tense in tenses:
    for gender in ['male', 'female']:
        third_person_combinations.append(('third', gender, 'neutral', tense))
    third_person_combinations.append(('third', 'they', 'neutral', tense))

# Merge all
valid_combinations = (
    first_person_combinations +
    second_person_combinations +
    third_person_combinations
)

# Example function to generate a prompt for a given combination
def generate_prompt(target_language, politeness, gender, person, tense, num_verbs=80):
    # return (
    #     f"Generate a python list of {num_verbs} distinct, unique and various verbs in {target_language} that are in the form of:\n"
    #     f"- Tense: {tense}\n"
    #     f"- Person: {person} person\n"
    #     f"- Gender: {gender}\n"
    #     f"- Politeness: {politeness} form\n"
    #     f"Provide only the python list of conjugated verbs in {target_language} only without translations or explanations."
    # )
    polite_note = f" using the '{politeness}' form" if person == 'second' else ""
    gender_note = "" if gender in ['neutral', 'they'] else f" for {gender} subject"
    return (
        f"Generate a list of {num_verbs} unique verbs in {target_language} that are in the form of:\n"
        f"- Tense: {tense}\n"
        f"- Person: {person} person{polite_note}{gender_note}.\n"
        f"Provide only the python list of conjugated verbs in {target_language} only without translations or explanations. Give gramatically correct phrases for the mentioned gender and tense.\n"
    )

# Example of generating prompts for all combinations
target_language = "Hindi"  # or any language of your choice
prompts = [
    {
        "combination": combo,
        "prompt": generate_prompt(target_language, *combo)
    }
    # for combo in combinations
    for combo in valid_combinations
]
print("Total prompts generated: ", len(prompts))
# Example: print a few prompts
for i in range(3, 6):
    print(f"Prompt for combination {prompts[i]['combination']}:\n{prompts[i]['prompt']}\n{'-'*80}\n")


In [ ]:
print("Length of prompts: ", len(prompts))
prompts[6:8]

In [ ]:
for model_path in model_ids_gemma:
    tokenizer, model, processor = model_init(model_path)
    for prompt in tqdm.tqdm(prompts):
        topic = prompt['combination']
        now_str = get_now_str()
        start_time = dt.datetime.strptime(now_str, datetime_format)
        save_logs(now_str, 'Model: '+model_path + '\nTopic: '+ str(topic))
        # messages = get_message(examples= examples, topic=topic)
        messages = get_message(tgt_lang=TGT_LANG,  topic=topic,   prompt=prompt['prompt'], examples= examples , prompt_type=None)
        inputs = get_prompt_inputs(processor, messages, model)
        now_str2 = get_now_str()
        end_time = dt.datetime.strptime(now_str2, datetime_format)
        save_logs(now_str2, messages, inputs, time_diff=  end_time-start_time)
        start_time = end_time
        decoded = get_decoded_response(model, processor, inputs)
        now_str3 = get_now_str()
        end_time = dt.datetime.strptime(now_str3, datetime_format)
        save_logs(now_str3, decoded=decoded, time_diff= end_time-start_time)
        # extracted_txt = get_extracted_text(decoded)
        effective_txt = get_effective_text(decoded, combo=topic)
        now_str = get_now_str()
        sav_outputs(now_str, effective_txt)

In [ ]:
break all the code into functions and run the code in a loop for all the models and topics

## Mistral:

In [ ]:
# Change this to the model you need to use. Use the above to see the list of models.
model_name_or_path = "/assets/models/mistralai-mistral-instruct-7b-v0.3"

In [ ]:
# %pip install protobuf
# %pip install sentencepiece

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name_or_path)
print("tokenizer loaded")

# LLaMa's tokenizer does not have a valid PAD token, so we need to initialize this as so
# tokenizer.pad_token = tokenizer.eos_token

# For decoder-only models, just to be safe, also do:
tokenizer.padding_side = "left"

In [ ]:
tokenizer

In [ ]:
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    # By default, map different parts of the model to available GPU(s).
    device_map="auto",
    # Loading the model in full precision can use a lot of
    # of memory, so we quantize it using reduced precision types.
    torch_dtype='bfloat16'
)

# Best practices
model.eval()

### Generation

More information on using the transformers library and its components can be found here: https://huggingface.co/docs/transformers/llm_tutorial

Specifically, for text generation, the following can be useful:
- https://huggingface.co/docs/transformers/main/en/main_classes/text_generation
- https://huggingface.co/blog/how-to-generate

In [ ]:
TASK_PROMPT = '''Generate 10 English sentences each for the relations: 
grandmother, grandfather, uncle, aunt, brother-in-law, sister-in-law, cousin, nephew, niece. 
For each relation, generate 10 sentences using the following topics: games, deep talks, questions, exclamations, and other forms of speeches. 
Ensure the sentences include different forms of possessive pronouns (e.g., my, their, his, her) for each of the 9 relations. Avoid using short/colloquial terms for the relations.
Only provide the response as a Python list of strings for all categories. No need to segregate them in any manner.
Sample output: 
<<<[
"My grandmother and I play chess together.", 
...
"Their grandfather and I have deep talks.", 
...
"Due to the heartache, my brother-in-law was not present in the function.",
...
"For the party, my uncle will pick and drop you.",
...
"I have the best sister-in-law in the world!",
...
"I am very grateful to my aunt to take care of my mother when I was not available",
...
... 
] (total 90 sentences: for each relation -> 10 sentences)>>>'''

def generate_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt")

    with torch.inference_mode():
        outputs = model.generate(
            **inputs.to(model.device),
            temperature=0.7,
            do_sample=True,
            num_return_sequences=2,
            num_beams=5,
            max_new_tokens=1000
        )

        outputs = tokenizer.batch_decode(
            outputs, skip_special_tokens=True,
            clean_up_tokenization_spaces=True
        )
    return outputs



In [ ]:
# for output in outputs:
#     print("o/p::", output)
    # print('-' * 50)
    # print()
# save the outputs to a file named : generated_outputs.txt
# open in append mode

def save_outputs(outputs):
    with open('generated_outputs_hin_mf.txt', 'a') as f:
        for output in outputs:
            f.write(output + '\n')
            # print(output)






In [ ]:
def extract_sentences(outputs, prompt):
    response = outputs[0]
    sentences = response[response.find(prompt) + len(prompt):]
    sentences = sentences[sentences.find('{') :sentences.find('}')]  
    print(sentences)
    # sentences = sentences.split(',\n')
    # sentences = [sentence.strip() for sentence in sentences]
    # sentences = [sentence[1:-1] for sentence in sentences]
    # print(sentences, type(sentences))
    return sentences

# extract_sentences(outputs)

In [ ]:
# save the sentences to a file named : generated_sentences.txt
def save_sentences(sentences):
    with open('generated_sentences_hin_mf.txt', 'a+') as f:
        # for sentence in sentences:
            f.write(sentences + '\n')


## Prompt template:


In [ ]:
prompts=[]
# techniques = examples.keys()
topics=["occupation", " religion", " sport", " politics", "  health", " finance", " education", " farming", " entertainment", "  news", "  daily conversation", "  weather", " technology", " conflicts", " controversials", 
        " international", "  UN", " travel", " tourism", " shopping", " baby care", " valentines", "  soldiers", " prisioners", " soul actions", " nature", " pollution", " bio hazards", " elders", " family", 
        "  social studies", " maths", " literature", "  physics", " chemistry", "  biology", " Indian history ", " civics", " geography", "  computer", " physical education", "  arts and craft", "  food", 
        " clothes", " water shortage", " road blockage", " traffic", " arriving late", " bargaining", " toys", " games", " deep talks", "declarative", "interrogative", "imperative",  "exclamatory", "safe work space", 
        "medical checkups", "self-reliant", "future India"]

In [ ]:
topics = [ "legal", "governance",  
  "STEM" ,   "business", "sports",  "culture", "Alarm",
    "Audio",
    "Calendar / Events",  # Added space for better readability
    "Communication",
    "DateTime",
    "Email",
    "Finance",  # Likely, but limited information available
    "General",
    "Home Automation",  # Likely, but limited information available
    "Location",
    "Music",
    "News",
    "Reminders",
    "Social Media",  # Likely, but limited information available
    "Travel",
    "Weather",
    "BookRestaurant",
    "BookFlight",
    "BookHotel",
    "GetDirections",
    "GetPlaceDetails",
    "GetWeather",
    "SearchForInformation",
    "GetNews",
    "PlayMusic",
    "ControlVolume",
    "SetAlarm",
    "CancelAlarm",
    "SetReminder",
    "MakeCall",
    "SendMessage",
    "SendEmail",
    "CheckEmail",
    "GetTime",
    "GetDate",
    "Greeting",
    "Farewell",
    "AskHowAreYou",
    "ThankYou",
    "TellAJoke",
    "TurnOnLights",  # and other home automation actions
    "Social media interactions",  # posting, liking, etc.
    "Financial transactions"  ]

topics=topics[:3]
len(topics)

In [ ]:

# for technique in techniques:
    # pick every 10 topics from topics in a loop
for i in tqdm.tqdm(range(0, len(topics))):
    # print(topics[i:i+10])
    # print("Generate 100 sentences in "+ lang +" with action doer as Male or Female. Differentiate between male and female sentences using the "+ technique +" technique. You can use the topics like "+ ", ".join(topics[i:i+10]) +".Return as a python dictionary.")
    # prompts.append("Generate 10 sentences in "+ lang +" with action doer as Male or Female. Differentiate between male and female sentences using the "+ technique +" technique. You can use the topics like "+ ", ".join(topics[i:i+10]) +".Return as a python dictionary.")
#     prompt = '''Generate 10 English sentences each for the relations: 
# grandmother, grandfather, uncle, aunt, brother-in-law, sister-in-law, cousin, nephew, niece. 
# For each relation, generate 10 sentences using the following topic: ''' + topics[i]+''' 
# Ensure the sentences include different forms of possessive pronouns (e.g., my, their, his, her) for each of the 9 relations. Avoid using short/colloquial terms for the relations.
# Only provide the response as a Python list of strings for all categories. No need to segregate them in any manner.
# Sample output: 
# <<<[
# "My grandmother and I play chess together.", 
# ...
# "Their grandfather and I have deep talks.", 
# ...
# "Due to the heartache, my brother-in-law was not present in the function.",
# ...
# "For the party, my uncle will pick and drop you.",
# ...
# "I have the best sister-in-law in the world!",
# ...
# "I am very grateful to my aunt to take care of my mother when I was not available",
# ...
# ... 
# ] (total 90 sentences: for each relation -> 10 sentences)>>>'''  
    prompt = '''Generate 10 Hindi sentences such that subject is male and female individually. Give the gender as key: 0 for Male and 1 for female.
      For each gender, generate 5 sentences each for following topic: ''' + topics[i]+''' 
Only provide the response as a Python list of strings for all categories. No need to segregate them in any manner.
Sample output: 
<<<{
 0: [  
            "लेखक नई पुस्तक लिख रहा है।",  
            ...
        ],  
        1: [  
            "लेखिका नई पुस्तक लिख रही है।",  
            ... 
        ]  
} (total 90 sentences: for each relation -> 10 sentences)>>>''' 
    prompts.append(prompt)

    outputs= generate_response(prompt=prompt)
    print("outputs done for topic: ", topics[i], i)
    save_outputs(outputs)
    sentences= extract_sentences(outputs, prompt)
    print("sentences extracted for topic:", topics[i], i)
    save_sentences(sentences)

        

For RLHF'd models (LLaMa-3.1 Instruct, etc.), an additional prompt formatting step is needed to ensure that the model is able to generate the desired output. The template is applied using `tokenizer.apply_chat_template` function, and basically adds formatting tokens to your prompt. Use it only with instruction-fine-tuned models.

In [ ]:
# # %pip install jinja2>=3.1.0

# TASK_PROMPT = "Please answer my question. What is the capital of India?"
# TASK_CONVERSATION = [
#     # System Prompt: This is optional, and not all models support this.
#     # But use it if you need explicit instructions to be followed.
#     dict(role='system', content='You are a helpful assistant.'),
#     # Your message (as if on the web interface) goes here.
#     # Past history can be added to this conversation too.
#     dict(role='user', content=TASK_PROMPT)
# ]

# # Format the conversation to a text prompt, using apply chat template.
# conversation_prompt = tokenizer.apply_chat_template(
#     TASK_CONVERSATION,
#     tokenize=False,
#     # Needed to allow the model to start its reply instead of completing yours.
#     add_generation_prompt=True
# )
# # We skip special tokens because the template already adds them. This is an overlooked thing, so be careful.
# inputs = tokenizer(conversation_prompt, return_tensors="pt", add_special_tokens=False)

# # Generation process is the same as before.
# with torch.inference_mode():
#     outputs = model.generate(
#         **inputs.to(model.device),
#         temperature=0.2,
#         do_sample=True,
#         num_return_sequences=2,
#         num_beams=2,
#         max_new_tokens=10
#     )

#     outputs = tokenizer.batch_decode(
#         outputs, skip_special_tokens=True,
#         clean_up_tokenization_spaces=True
#     )

#     for output in outputs:
#         print(output)
#         print('-' * 50)
#         print()

# Conclusion: